In [ ]:
# 从 GitHub 安装源码：使用 git+https 链接。
# 从本地源码安装：在源码目录下使用 pip install .。
# 开发模式安装：使用 pip install -e .

In [2]:
!pip install -q git+https://github.com/tensorflow/docs

  Preparing metadata (setup.py) ... done


In [ ]:
# 包含 __init__.py 文件的目录是 Python 包，可以通过包的形式导入。
# 没有 __init__.py 的目录不是包，只是普通目录，不能作为包导入。
# 如果目录没有 __init__.py，你就不能像导入包一样导入它。你只能直接导入模块

In [4]:
import keras

from keras import layers
from keras import ops
from tensorflow_docs.vis import embed
import tensorflow as tf
import numpy as np
import imageio

In [ ]:
# RGB 通道的 0-255 范围表示颜色的强度，而不直接表示颜色的纯度。
# 颜色的纯度通常由饱和度来衡量，表示颜色是否是纯色（没有灰色成分）
# 彩色图像数字化的过程就是将图像中的每个像素分解为三个数值，分别表示红色、绿色和蓝色的强度。
# 每个通道保存了整个图像中该颜色的强度分布，可以通过这三个通道来重建图像。
# 最终图像的表示就是三个二维数组的集合，这些数组分别表示红色、绿色和蓝色通道的数据

In [3]:
batch_size = 64
num_channels = 1
num_classes = 10
image_size = 28
latent_dim = 128

In [4]:
# 准备数据
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_labels = np.concatenate([y_train, y_test])

# 归一化
all_digits = all_digits.astype("float32") / 255.0
all_digits = np.reshape(all_digits, (-1, 28, 28, 1)) # 变成四维张量
all_labels = keras.utils.to_categorical(all_labels, 10) # one-hot
dataset = tf.data.Dataset.from_tensor_slices((all_digits, all_labels))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)
print(f"Shape of training images: {all_digits.shape}")
print(f"Shape of training labels: {all_labels.shape}")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Shape of training images: (70000, 28, 28, 1)
Shape of training labels: (70000, 10)


In [5]:
# 计算生成器和鉴别器的输入通道数
# 在常规（无条件）GAN 中，我们首先从正态分布中采样噪声（某个固定维度）。在我们的例子中，我们还需要考虑类标签。我们必须将
# 类数添加到生成器（噪声输入）和鉴别器（生成的图像输入）的输入通道中。
# 生成器输入通道数（generator_in_channels）：由潜在空间的维度和类别数量的和构成，表示生成器接收的输入数据。
# 判别器输入通道数（discriminator_in_channels）：由图像的通道数和类别数量的和构成，表示判别器接收的输入数据
# 生成器的输入是由潜在向量（latent_dim）和类别信息（num_classes）组合而成的。生成器接收这两个信息，生成符合条件的图像或数据。
# 在条件 GAN 中，判别器不仅要判断图像是否真实，还需要判断图像是否符合给定的类别标签。
generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes
print(generator_in_channels, discriminator_in_channels)

138 11


In [5]:
layers.GlobalMaxPooling2D(  )

<GlobalMaxPooling2D name=global_max_pooling2d, built=False>

In [6]:
# Create the discriminator.
discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((28, 28, discriminator_in_channels)), # (28,28)
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"), # (14,14)
        layers.LeakyReLU(negative_slope=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"), # (7,7)
        layers.LeakyReLU(negative_slope=0.2),
        layers.GlobalMaxPooling2D(), # 它用于减少特征图的空间尺寸，同时保留每个通道中的最重要的信息。(b,d)
        layers.Dense(1),
    ],
    name="discriminator",
)

# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),  # 输入层：接收形状为 (generator_in_channels,) 的一维向量作为输入
        layers.Dense(7 * 7 * generator_in_channels), # 第一层全连接层，将输入的维度投影为一个 7x7xgenerator_in_channels 的向量
        layers.LeakyReLU(negative_slope=0.2),
        layers.Reshape((7, 7, generator_in_channels)), # 将一维向量 reshape 成 7x7xgenerator_in_channels 的 3D 张量，方便后续卷积操作
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"), # (14,14)
        layers.LeakyReLU(negative_slope=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),  # (28,28)
        layers.LeakyReLU(negative_slope=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"), # 最后一层卷积：生成单通道图像（如灰度图），使用 sigmoid 激活函数将输出压缩到 [0, 1] 之间
    ],
    name="generator",
)

In [7]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim): # 初始化方法，接收判别器、生成器和潜在空间维度
        super().__init__() # 调用父类的初始化方法
        self.discriminator = discriminator  # 判别器
        self.generator = generator # 生成器
        self.latent_dim = latent_dim # 潜在空间的维度
        self.seed_generator = keras.random.SeedGenerator(1337) # # 用于生成随机种子的生成器
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")  # 用于跟踪生成器损失的均值
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss") # 用于跟踪判别器损失的均值

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker] # 返回监控的指标，包括生成器和判别器的损失

    def compile(self, d_optimizer, g_optimizer, loss_fn): # 编译模型，设置判别器优化器、生成器优化器和损失函数
        super().compile()  # 调用父类的 compile 方法
        self.d_optimizer = d_optimizer  # 判别器优化器
        self.g_optimizer = g_optimizer # 生成器优化器
        self.loss_fn = loss_fn # 损失函数

    def train_step(self, data):
        # 解包数据：real_images 是真实图像，one_hot_labels 是图像的 one-hot 标签
        real_images, one_hot_labels = data
        # 为了能够与图像连接，需要为标签添加额外的维度（用于判别器的输入）
        image_one_hot_labels = one_hot_labels[:, :, None, None] # 添加额外的维度
        image_one_hot_labels = ops.repeat(
            image_one_hot_labels, repeats=[image_size * image_size] # 重复标签数据
        )
        image_one_hot_labels = ops.reshape(# 重塑形状
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )

        # 从潜在空间采样随机向量，并将标签与随机向量拼接，作为生成器的输入
        batch_size = ops.shape(real_images)[0]
        random_latent_vectors = keras.random.normal(
            shape=(batch_size, self.latent_dim), seed=self.seed_generator
        )
        random_vector_labels = ops.concatenate(  # 拼接标签与潜在向量
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # 使用生成器解码潜在向量（根据标签引导）生成虚假图像
        generated_images = self.generator(random_vector_labels)

        # 在特征轴合并通道特征
        fake_image_and_labels = ops.concatenate(
            [generated_images, image_one_hot_labels], -1
        )
        real_image_and_labels = ops.concatenate([real_images, image_one_hot_labels], -1)
        combined_images = ops.concatenate(  # 将虚假图像和真实图像拼接
            [fake_image_and_labels, real_image_and_labels], axis=0
        )

        # 组装判别标签，用于标识真假图像。这里虚假图像标签为 1，真实图像标签为 0,作者真是反常规
        # 因为判别器如果把1当假，那0就是真,这里组装后就是二分类问题了
        labels = ops.concatenate(
            [ops.ones((batch_size, 1)), ops.zeros((batch_size, 1))], axis=0
        )

        # 训练判别器
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)  # 判别器对合并后的图像进行预测
            d_loss = self.loss_fn(labels, predictions)  # 判别器的损失函数，计算真实标签与预测值的误差
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights) # 计算损失对判别器参数的梯度
        self.d_optimizer.apply_gradients( # 使用判别器优化器更新判别器的权重
            zip(grads, self.discriminator.trainable_weights)
        )

         # 再次从潜在空间采样随机向量，作为生成器输入
        random_latent_vectors = keras.random.normal(
            shape=(batch_size, self.latent_dim), seed=self.seed_generator
        )  # 随机生成潜在向量
        random_vector_labels = ops.concatenate( # 拼接标签与潜在向量
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # 为生成器创建标签，标识“所有生成的图像都是真实的”（为了欺骗判别器）
        misleading_labels = ops.zeros((batch_size, 1)) # 0表示真实,作者反向思维，其实判别器是二分类

        # 训练生成器（注意：这里不更新判别器的权重）
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels) # 使用生成器生成虚假图像
            fake_image_and_labels = ops.concatenate(  # 拼接虚假图像与标签
                [fake_images, image_one_hot_labels], -1
            )
            predictions = self.discriminator(fake_image_and_labels) # 判别器对虚假图像进行预测
            g_loss = self.loss_fn(misleading_labels, predictions) # 生成器的损失函数，计算真标签与判别器预测值的误差
        grads = tape.gradient(g_loss, self.generator.trainable_weights) # 计算损失对生成器参数的梯度
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights)) # 使用生成器优化器更新生成器的权重
        # 监控损失
        self.gen_loss_tracker.update_state(g_loss)  # 更新生成器损失的均值指标
        self.disc_loss_tracker.update_state(d_loss)  # 更新判别器损失的均值指标
        return { # 返回平均损失
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }

In [8]:
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

cond_gan.fit(dataset, epochs=20)

Epoch 1/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 41s 28ms/step - d_loss: 0.4525 - g_loss: 1.4650
Epoch 2/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 28s 25ms/step - d_loss: 0.5433 - g_loss: 1.1484
Epoch 3/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 28s 26ms/step - d_loss: 0.4516 - g_loss: 1.3488
Epoch 4/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 28s 25ms/step - d_loss: 0.2243 - g_loss: 2.7196
Epoch 5/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 41s 25ms/step - d_loss: 0.1714 - g_loss: 2.4838
Epoch 6/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 41s 26ms/step - d_loss: 0.5909 - g_loss: 1.2616
Epoch 7/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 28s 25ms/step - d_loss: 0.6470 - g_loss: 0.9631
Epoch 8/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 28s 25ms/step - d_loss: 0.6751 - g_loss: 0.8682
Epoch 9/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 42s 26ms/step - d_loss: 0.6759 - g_loss: 0.8172
Epoch 10/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 28s 26ms/step - d_loss: 0.6873 - g_loss: 0.7632
Epoch 11/20
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 28s 25ms/step - d_loss: 0.6892 - g_loss: 0.7569
Epoch 12

In [6]:
interpolation_noise = keras.random.normal(shape=(1, 8))
interpolation_noise = ops.repeat(interpolation_noise, repeats=6)

In [7]:
interpolation_noise

<tf.Tensor: shape=(48,), dtype=float32, numpy=
array([ 0.00776863,  0.00776863,  0.00776863,  0.00776863,  0.00776863,
        0.00776863,  0.7953589 ,  0.7953589 ,  0.7953589 ,  0.7953589 ,
        0.7953589 ,  0.7953589 , -1.4645193 , -1.4645193 , -1.4645193 ,
       -1.4645193 , -1.4645193 , -1.4645193 , -0.42894912, -0.42894912,
       -0.42894912, -0.42894912, -0.42894912, -0.42894912,  0.31918424,
        0.31918424,  0.31918424,  0.31918424,  0.31918424,  0.31918424,
        1.3588815 ,  1.3588815 ,  1.3588815 ,  1.3588815 ,  1.3588815 ,
        1.3588815 ,  0.04990932,  0.04990932,  0.04990932,  0.04990932,
        0.04990932,  0.04990932, -0.35721895, -0.35721895, -0.35721895,
       -0.35721895, -0.35721895, -0.35721895], dtype=float32)>

In [9]:
# 这段代码的目的是通过 条件生成对抗网络（Conditional GAN） 实现类间插值，生成
# 从一个类别到另一个类别的平滑过渡图像。它通过对类标签进行插值，并结合噪声向量，
# 生成过渡图像。
trained_gen = cond_gan.generator # 获取训练好的生成器

# num_interpolation 表示在两个类别的开始图像（first_number）和结束图像（s
# econd_number）之间生成的插值图像数量。总图像数为 num_interpolation + 2，
# 因为除了插值的图像外，还包括了起始图像和结束图像。
num_interpolation = 9

# interpolation_noise 是一个包含随机噪声的张量，用于生成插值图像。
# 首先生成一个形状为 (1, latent_dim) 的噪声向量，这个噪声向量是从标准正态分布
# 中随机采样的。latent_dim 是潜在空间的维度（也就是生成器输入噪声的维度）。
interpolation_noise = keras.random.normal(shape=(1, latent_dim))
# 然后使用 ops.repeat 将这个噪声向量复制 num_interpolation 次，得到一个形状为 (
#     num_interpolation, latent_dim) 的噪声矩阵。
interpolation_noise = ops.repeat(interpolation_noise, repeats=num_interpolation)
interpolation_noise = ops.reshape(interpolation_noise, (num_interpolation, latent_dim))

# interpolate_class 函数的目的是进行两个类别之间的标签插值。
# first_number 和 second_number 是表示类别的整数（例如：2 和 6）。这两个数字将被转换为 one-hot 编码 格式。
# 使用 keras.utils.to_categorical 将这两个类别数字转换为 one-hot 编码向量。num_classes 是类别总数（
# 例如，对于 MNIST，是 10 类）。
# ops.cast 将 one-hot 编码的标签转换为 float32 类型，确保数据类型一致。
def interpolate_class(first_number, second_number):
    # Convert the start and end labels to one-hot encoded vectors.
    first_label = keras.utils.to_categorical([first_number], num_classes)
    second_label = keras.utils.to_categorical([second_number], num_classes)
    first_label = ops.cast(first_label, "float32")
    second_label = ops.cast(second_label, "float32")

    # percent_second_label 是一个插值向量，它在 [0, 1] 范围内等间隔地生成
    percent_second_label = ops.linspace(0, 1, num_interpolation)[:, None]
    percent_second_label = ops.cast(percent_second_label, "float32")
    interpolation_labels = (
        first_label * (1 - percent_second_label) + second_label * percent_second_label
    )

    # Combine the noise and the labels and run inference with the generator.
    noise_and_labels = ops.concatenate([interpolation_noise, interpolation_labels], 1)
    fake = trained_gen.predict(noise_and_labels)
    return fake

# start_class 和 end_class 分别表示插值的起始和结束类别。在这个例子中，start_class 是 2，
# end_class 是 6。
start_class = 0
end_class = 6
# 调用 interpolate_class 函数，生成从 start_class 到 end_class 之间的平滑过渡图像，并将结果存储在 fake_images 中。
fake_images = interpolate_class(start_class, end_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 663ms/step


In [ ]:
fake_images *= 255.0
converted_images = fake_images.astype(np.uint8)
converted_images = ops.image.resize(converted_images, (96, 96)).numpy().astype(np.uint8)
imageio.mimsave("animation.gif", converted_images[:, :, :, 0], fps=1)
embed.embed_file("animation.gif")